<a href="https://colab.research.google.com/github/Agnes-OSNA/ADM-project/blob/master/Bert_with_wiki_words_no_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla P100-PCIE-16GB'

In [2]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 9.2MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 


In [11]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving all_files.txt to all_files.txt


In [18]:
df = pd.read_csv("all_files.txt", delimiter='\n', header=None)
df.sample(5)

,0
31,I agree with you - Students do need to learn h...
7,"First off, college is nothing like the movies...."
26,"I, for one, do not view anarchism as an end, o..."
11,"Not beyond class discussion, and even thats ra..."
32,"I believe that most college courses, while int..."


In [65]:
from pytorch_pretrained_bert import BertTokenizer
import nltk
nltk.download('punkt')

sentences = []
for key, value in df.iteritems(): 
    #print(value)
    for i, e in enumerate(value):
      temp = nltk.tokenize.sent_tokenize(e)
      sentences = sentences + temp
print("\nFirst sentence: ", sentences[0])
print("Number of sentences: ", len(sentences))

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
print("\nFirst sentence: ", sentences[0])
print("Wiki sentence: ", sentences[1])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("\nFirst sentence:", tokenized_texts[0])
print ("Wiki sentence:", tokenized_texts[1])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

First sentence:  I am also currently seeking my degree from said institution and I am shocked and awed everyday by the mindlessness of the students.
Number of sentences:  909

First sentence:  [CLS] I am also currently seeking my degree from said institution and I am shocked and awed everyday by the mindlessness of the students. [SEP]
Wiki sentence:  [CLS] I attend a liberal arts college which does contain a large percentage of students seeking change, but who are not heard or become distracted by  Wiki__Sesame__azfzo  Tequilla. [SEP]

First sentence: ['[CLS]', 'i', 'am', 'also', 'currently', 'seeking', 'my', 'degree', 'from', 'said', 'institution', 'and', 'i', 'am', 'shocked', 'and', 'awe', '##d', 'everyday', 'by', 'the', 'mind', '##lessness', 'of', 'the', 'students', '.', '[SEP]']
Wiki sentence: ['[CLS]', 'i', 'attend', 'a', 'liberal', 'arts', 'college', 'which', 'does', 'c

In [58]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 512

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1119 > 512). Running this sequence through BERT will result in indexing errors


**Question**
Should we cut the sentences? But lots of wiki works will be lost... Or cut sentences into chunks?

**Question**
Could we remove "wikification" to get embeddings? Or could we replace words in vocabulary file?